In [ ]:
# define and move to dataset directory
datasetdir = 'E:/bangkit2021/dataset/Plant_leaf_diseases_dataset_with_augmentation'
import os
os.chdir(datasetdir)

# import the needed packages
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
print('Num GPU Available: ', len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 30

def generators(shape, preprocessing): 
    '''Create the training and validation datasets for 
    a given image shape.
    '''
    imgdatagen = ImageDataGenerator(
        preprocessing_function = preprocessing,
        horizontal_flip = True, 
        validation_split = 0.1)

    height, width = shape

    train_dataset = imgdatagen.flow_from_directory(
        os.getcwd(),
        target_size = (height, width),
        batch_size = batch_size,
        subset = 'training')

    val_dataset = imgdatagen.flow_from_directory(
        os.getcwd(),
        target_size = (height, width),
        batch_size = batch_size,
        subset = 'validation')
    
    return train_dataset, val_dataset

In [ ]:
def plot_history(history, yrange):
    '''Plot loss and accuracy as a function of the epoch,
    for the training and validation datasets.
    '''
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Get number of epochs
    epochs = range(len(acc))

    # Plot training and validation accuracy per epoch
    plt.plot(epochs, acc)
    plt.plot(epochs, val_acc)
    plt.title('Training and validation accuracy')
    plt.ylim(yrange)
    
    # Plot training and validation loss per epoch
    plt.figure()

    plt.plot(epochs, loss)
    plt.plot(epochs, val_loss)
    plt.title('Training and validation loss')
    
    plt.show()

In [ ]:
# Define a Callback class that stops training once accuracy reaches 97.0%
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.97):
            print("\nReached 97.0% accuracy so cancelling training!")
            self.model.stop_training = True

__VGG19__

In [ ]:
vgg19 = keras.applications.vgg19
conv_model = vgg19.VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))
for layer in conv_model.layers: 
    layer.trainable = False
x = keras.layers.Flatten()(conv_model.output)
x = keras.layers.Dense(64, activation='relu')(x)
x = keras.layers.Dense(128, activation='relu')(x)
predictions = keras.layers.Dense(39, activation='softmax')(x)
full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)
full_model.summary()

In [ ]:
train_dataset, val_dataset = generators((224,224), preprocessing=vgg19.preprocess_input)

In [ ]:
full_model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adamax(learning_rate=0.001),
                  metrics=['acc'])

In [ ]:
callbacks = myCallback()
history = full_model.fit(
    train_dataset, 
    validation_data = val_dataset,
    epochs=200,
    callbacks = [callbacks])

In [ ]:
plot_history(history, yrange=(0.9,1))

In [ ]:
# save the model in SavedModel format
full_model.save("model")